In [ ]:
#graph with the subplots for each year
# mobility_trips_years = mobility_trips_motive[years]

# # Melt the data to create a 'year' column
# mobility_trips_years = mobility_trips_years.reset_index().melt(id_vars=['motive'], var_name='year', value_name='trips')

# fig = px.bar(mobility_trips_years, x='trips', y='motive', color='motive', facet_col='year', orientation='h', hover_name='trips', width=1100, height=400)

# fig.update_layout(title={
#     'text': 'Average amount of trips for every vehicle combined per day from 2018 to 2022',
#     'x': 0.5
# })
# fig.update_yaxes(title_text="Travel motives")
# fig.show()